In [1]:
import tensorflow as tf
import  numpy as np
from glob import glob
from scipy import misc
import os, cv2, random
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import ticker


In [2]:
class DogsAndCatsHelper:

    @staticmethod
    def get_data():
        
        TRAIN_DIR = 'C:/Users/Xiangyu Gao/Downloads/train/'
        TEST_DIR = 'C:/Users/Xiangyu Gao/Downloads/test1/'

        ROWS = 227
        COLS = 227
        CHANNELS = 3
        print(os.listdir(TRAIN_DIR)[:5])
        train_images = [TRAIN_DIR+i for i in os.listdir(TRAIN_DIR)] # use this for full dataset
        train_dogs =   [TRAIN_DIR+i for i in os.listdir(TRAIN_DIR) if 'dog' in i]
        train_cats =   [TRAIN_DIR+i for i in os.listdir(TRAIN_DIR) if 'cat' in i]

        test_images =  [TEST_DIR+i for i in os.listdir(TEST_DIR)]


        # slice datasets for memory efficiency on Kaggle Kernels, delete if using full dataset
        train_images = train_dogs[:] + train_cats[:]
        random.shuffle(train_images)
        test_images =  test_images[:]

        def read_image(file_path):
            img = cv2.imread(file_path, cv2.IMREAD_COLOR) #cv2.IMREAD_GRAYSCALE
            return cv2.resize(img, (ROWS, COLS), interpolation=cv2.INTER_CUBIC)


        def prep_data(images):
            count = len(images)
            data = np.ndarray((count, CHANNELS, ROWS, COLS), dtype=np.uint8)

            for i, image_file in enumerate(images):
                image = read_image(image_file)
                data[i] = image.T
                if i%250 == 0: print('Processed {} of {}'.format(i, count))
            
            return data

        train = prep_data(train_images[0:5000])
        test = prep_data(test_images[0:5000])
        train = np.swapaxes(train, 1,3)
        test = np.swapaxes(test, 1,3)

        print("Train shape: {}".format(train.shape))
        print("Test shape: {}".format(test.shape))
        labels = []
        
        for i in train_images:
            if 'dog' in i.split('/')[-1]:
                labels.append([1,0])
            else:
                labels.append([0,1])

        #sns.countplot(labels).set_title('Cats and Dogs')
        labels = np.array(labels[0:5000])

        return train, labels, test

In [8]:
class AlexNet:

    def __init__(self,
                 batch_size = 32,
                 epochs = 100):
        """
        Args:
            batch_size: number of examples per batch_size
            epochs: number of iterations over all training examples
        """
        self.batch_size = batch_size
        self.epochs = epochs

    @staticmethod
    def inference(images, num_classes):
        """Builds AlexNet Graph

        Args:
            images: train/test images
            num_classes: number of classes
        Returns:
            final_node: last node in the graph
        """

        # Layer 1
        with tf.name_scope("conv1") as scope:
            kernel = AlexNetHelper.instintiate_weights('W1', [11, 11, 3, 96])
            conv = tf.nn.conv2d(images, kernel, [1, 4, 4, 1], padding='SAME')
            biases = AlexNetHelper.instintiate_bias('b1', [96])
            conv1 = tf.nn.relu(tf.nn.bias_add(conv, biases), name=scope)

        with tf.name_scope('batch_norm1') as scope:
            mean, var = tf.nn.moments(conv1, [0, 1, 2])
            batch_norm1 = tf.nn.batch_normalization(conv1, mean, var, 0, 1, 0, name=scope)

        with tf.name_scope('pool1') as scope:
            pool1 = tf.nn.max_pool(batch_norm1, [1, 3, 3, 1], [1, 2, 2, 1], padding='SAME', name=scope)

        # Layer 2
        with tf.name_scope("conv2") as scope:
            kernel = AlexNetHelper.instintiate_weights('W2', [5, 5, 96, 256])
            conv = tf.nn.conv2d(pool1, kernel, [1, 1, 1, 1], padding='SAME')
            biases = AlexNetHelper.instintiate_bias('b2', [256])
            conv2 = tf.nn.relu(tf.nn.bias_add(conv, biases), name=scope)

        with tf.name_scope('batch_norm2') as scope:
            mean, var = tf.nn.moments(conv2, [0, 1, 2])
            batch_norm2 = tf.nn.batch_normalization(conv2, mean, var, 0, 1, 0, name=scope)

        with tf.name_scope('pool2') as scope:
            pool2 = tf.nn.max_pool(batch_norm2, [1, 3, 3, 1], [1, 2, 2, 1], padding='SAME', name=scope)

        # Layer 3
        with tf.name_scope("conv3") as scope:
            kernel = AlexNetHelper.instintiate_weights('W3', [3, 3, 256, 384])
            conv = tf.nn.conv2d(pool2, kernel, [1, 1, 1, 1], padding='SAME')
            biases = AlexNetHelper.instintiate_bias('b3', [384])
            conv3 = tf.nn.relu(tf.nn.bias_add(conv, biases), name=scope)

        with tf.name_scope('batch_norm3') as scope:
            mean, var = tf.nn.moments(conv3, [0, 1, 2])
            batch_norm3 = tf.nn.batch_normalization(conv3, mean, var, 0, 1, 0, name=scope)

        # Layer 4
        with tf.name_scope("conv4") as scope:
            kernel = AlexNetHelper.instintiate_weights('W4', [3, 3, 384, 384])
            conv = tf.nn.conv2d(batch_norm3, kernel, [1, 1, 1, 1], padding='SAME')
            biases = AlexNetHelper.instintiate_bias('b4', [384])
            conv4 = tf.nn.relu(tf.nn.bias_add(conv, biases), name=scope)

        with tf.name_scope('batch_norm4') as scope:
            mean, var = tf.nn.moments(conv4, [0, 1, 2])
            batch_norm4 = tf.nn.batch_normalization(conv4, mean, var, 0, 1, 0, name=scope)

        # Layer 5
        with tf.name_scope("conv5") as scope:
            kernel = AlexNetHelper.instintiate_weights('W5', [3, 3, 384, 256])
            conv = tf.nn.conv2d(batch_norm4, kernel, [1, 1, 1, 1], padding='SAME')
            biases = AlexNetHelper.instintiate_bias('b5', [256])
            conv5 = tf.nn.relu(tf.nn.bias_add(conv, biases), name=scope)

        with tf.name_scope('batch_norm5') as scope:
            mean, var = tf.nn.moments(conv5, [0, 1, 2])
            batch_norm5 = tf.nn.batch_normalization(conv5, mean, var, 0, 1, 0, name=scope)

        with tf.name_scope('pool5') as scope:
            pool5 = tf.nn.max_pool(batch_norm5, [1, 3, 3, 1], [1, 2, 2, 1], padding='SAME', name=scope)

        # Fully Connected 6
        with tf.name_scope('FC6') as scope:
            pool5_flat = tf.contrib.layers.flatten(pool5)
            fc6 = tf.layers.dense(pool5_flat, units=4096, activation=tf.nn.relu, name=scope)
            # fc6_dropout = tf.layers.dropout(fc6, 0.5)

        with tf.name_scope('FC7') as scope:
            fc7 = tf.layers.dense(fc6, units=4096, activation=tf.nn.relu, name=scope)
            # fc7_dropout = tf.layers.dropout(fc7, 0.5)

        with tf.name_scope('classes') as scope:
            predictions = tf.layers.dense(fc7, units=num_classes, activation=tf.nn.softmax, name=scope)

        return predictions

    def run(self):
        """
        Runtime for AlexNet
        :param
        train_data: training data
        num_classes: num_classes
        :return: None
        """

        train_images, train_labels, valid_images= DogsAndCatsHelper.get_data()
        num_classes = train_labels.shape[1]

        images = tf.placeholder(tf.float32, shape=[None, *train_images[0].shape])
        ph_labels = tf.placeholder(tf.float32, shape=[None, 2])

        predictions = self.inference(images, num_classes)
        init = tf.global_variables_initializer()
        
        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits = predictions, labels = ph_labels))
        #loss = tf.nn.l2_loss(tf.square(predictions - labels))
        learner = tf.train.AdamOptimizer(0.001)
        optimizer = learner.minimize(loss)

        correct_prediction = tf.equal(tf.argmax(predictions, 1), tf.argmax(ph_labels, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

        with tf.Session() as sess:
            sess.run(init)
            for _ in range(self.epochs):
                l_total = 0
                acc = 0
                for i in range(0, len(train_images), self.batch_size):
                    l, acc, _ = sess.run(fetches=[loss, accuracy, optimizer],
                                         feed_dict={images: train_images[i: self.batch_size + i],
                                                    ph_labels: train_labels[i: self.batch_size + i]})
                    l_total += np.sum(l)

                    print(l_total, acc)

In [4]:
class AlexNetHelper:
    @staticmethod
    def instintiate_weights(key, shape):
        return tf.get_variable(key,
                               shape=shape,
                               initializer=tf.contrib.layers.variance_scaling_initializer(factor=2.0,
                                                                                          mode='FAN_IN',
                                                                                          uniform=False,
                                                                                          seed=None,
                                                                                          dtype=tf.float32))

    @staticmethod
    def instintiate_bias(key, shape):
        return tf.Variable(tf.constant(0.1, dtype=tf.float32, shape=shape))

In [11]:
from AlexNet import AlexNet

alex_net = AlexNet()
alex_net.run()

['cat.0.jpg', 'cat.1.jpg', 'cat.10.jpg', 'cat.100.jpg', 'cat.1000.jpg']


MemoryError: 

In [10]:
tf.reset_default_graph()